In [1]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import spacy
import re

In [2]:
nlp = spacy.load('en_core_web_sm')

## Data Loading, Preprocessing and Embeddings 

In [3]:
df = pd.read_csv('data/Tweets.csv')
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
lemmatizer = WordNetLemmatizer()

# since these are tweets, remove the @s to the airline and other users
def remove_statics(text):
    text = re.sub('@([A-Za-z0-9_]+)', '' , text)
    return text

def remove_punctuations(text):
    text = str(text)
    return text.translate(str.maketrans('' , '' , string.punctuation)).lower()

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    text = text.split()
    return ' '.join([e for e in text if e not in stop_words])

def lemmatize(text):
    lemmatized_text = []
    for word in text.split():
        lemmatized_text.append(lemmatizer.lemmatize(word))
    return ' '.join(lemmatized_text)

def pos_tag(tokenized_text):
    tokenized_text = tokenized_text.split()
    return nltk.pos_tag(tokenized_text)

In [5]:
df['preprocessed_text'] = df['text'].apply(remove_stopwords)
df['preprocessed_text'] = df['preprocessed_text'].apply(remove_statics)
df['preprocessed_text'] = df['preprocessed_text'].apply(lemmatize)
df['preprocessed_text'] = df['preprocessed_text'].apply((remove_punctuations))
df['text_pos'] = df['text'].apply(pos_tag)
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,preprocessed_text,text_pos
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),what said,"[(@VirginAmerica, VB), (What, WP), (@dhepburn,..."
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),plus added commercial experience tacky,"[(@VirginAmerica, JJ), (plus, CC), (you've, NN..."
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),i today must mean i need take another trip,"[(@VirginAmerica, NN), (I, PRP), (didn't, VBP)..."
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),really aggressive blast obnoxious entertainmen...,"[(@VirginAmerica, NN), (it's, NN), (really, RB..."
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),really big bad thing,"[(@VirginAmerica, NN), (and, CC), (it's, VB), ..."


### Encoding Labels

In [6]:
from sklearn import preprocessing

In [7]:
labels = df['airline_sentiment'].unique()
print(labels)

['neutral' 'positive' 'negative']


In [8]:
le = preprocessing.LabelEncoder()
encoded_labels = le.fit(labels)
print(encoded_labels.classes_)

['negative' 'neutral' 'positive']


### Word2Vec Vocab and Embedding Matrix

In [9]:
from gensim.models import Word2Vec

In [10]:
utterances = []
for u in df['preprocessed_text']:
    utterances.append(u.split())

In [11]:
w2v = Word2Vec(sentences=utterances, vector_size=50, window=5, min_count=65, workers=4, sg=1)
w2v.save('word2vec_model_1.wv')

In [12]:
for index, word in enumerate(w2v.wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(w2v.wv.index_to_key)} is {word}")

word #0/446 is i
word #1/446 is flight
word #2/446 is get
word #3/446 is thanks
word #4/446 is cancelled
word #5/446 is service
word #6/446 is hour
word #7/446 is time
word #8/446 is you
word #9/446 is customer


In [13]:
df.iloc[0]['text']

'@VirginAmerica What @dhepburn said.'

In [14]:
df = df.filter(['preprocessed_text' , 'airline_sentiment'])
df.dropna

<bound method DataFrame.dropna of                                        preprocessed_text airline_sentiment
0                                              what said           neutral
1                 plus added commercial experience tacky          positive
2             i today must mean i need take another trip           neutral
3      really aggressive blast obnoxious entertainmen...          negative
4                                   really big bad thing          negative
...                                                  ...               ...
14635                 thank got different flight chicago          positive
14636  leaving 20 minute late flight no warning commu...          negative
14637        please bring american airlines blackberry10           neutral
14638  money change flight answer phones any suggesti...          negative
14639  8 ppl need 2 know many seat next flight plz pu...           neutral

[14640 rows x 2 columns]>

## CNN Model 1

In [15]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import numpy as np

In [16]:
class Tweets(Dataset):
    def __init__(self, df):
        self.df = df
        self.label_map = {'neutral' : 0 , 'positive': 1 , 'negative':2}
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        text = self.df.iloc[idx]['preprocessed_text']
        label = self.df.iloc[idx]['airline_sentiment']
        return self.label_map[label] , text

In [17]:
tweets = Tweets(df)
train_iter = iter(Tweets(df))

batch_size = 64
train_dataloader = DataLoader(tweets , batch_size=batch_size)
test_dataloader = DataLoader(tweets , batch_size=batch_size)

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) # Remove -1 for labels to work from tutorial implementation

In [18]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list, text_list, offsets

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [19]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class ):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [20]:
train_iter = Tweets(df)
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class)

In [21]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [22]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = Tweets(df) , Tweets(df)
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  1.17s | valid accuracy    0.717 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  1.21s | valid accuracy    0.742 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.97s | valid accuracy    0.747 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.88s | valid accuracy    0.732 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.88s | valid accuracy    0.738 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.92s |